In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

In [4]:
import weaviate, os

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
        "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

True

## Data Preview

In [8]:
# Get the collection
movies = client.collections.get("Movies")

response = movies.query.fetch_objects(limit=5)

for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        f"https://www.themoviedb.org/movie/{o.properties['tmdb_id']}",
        o.properties["poster_url"]
    )

Aladdin 1992 https://www.themoviedb.org/movie/812 https://image.tmdb.org/t/p/w600_and_h900_bestv2/eLFfl7vS8dkeG1hKp5mwbm37V83.jpg
Terminator 2: Judgment Day 1991 https://www.themoviedb.org/movie/280 https://image.tmdb.org/t/p/w600_and_h900_bestv2/5M0j0B18abtBI5gi2RhfjjurTqb.jpg
RoboCop 2 1990 https://www.themoviedb.org/movie/5549 https://image.tmdb.org/t/p/w600_and_h900_bestv2/99l6Jt3ygKKmv7K8D6jdLz33tcB.jpg
Die Hard 2 1990 https://www.themoviedb.org/movie/1573 https://image.tmdb.org/t/p/w600_and_h900_bestv2/2rto4EhQrUsJoGGgWcwiRv6hB3H.jpg
Hot Shots! 1991 https://www.themoviedb.org/movie/9595 https://image.tmdb.org/t/p/w600_and_h900_bestv2/hoPS7RSvdZhZhWf28OfP2IZ0cb3.jpg


## Title search

In [9]:
import weaviate.classes.query as wq

movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="title",  # The target vector to search against
    limit=5,
    # return_properties=["title", "release_date", "tmdb_id", "poster_url"],
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

Home Alone 1990 771 https://image.tmdb.org/t/p/w600_and_h900_bestv2/onTSipZ8R3bliBdKfPtsDuHTdlL.jpg
Distance to query: 0.474

Home Alone 2: Lost in New York 1992 772 https://image.tmdb.org/t/p/w600_and_h900_bestv2/uuitWHpJwxD1wruFl2nZHIb4UGN.jpg
Distance to query: 0.499

Delicatessen 1991 892 https://image.tmdb.org/t/p/w600_and_h900_bestv2/gNtOgQHxE5B8e08zuNRAdDpmK5Z.jpg
Distance to query: 0.500

True Romance 1993 319 https://image.tmdb.org/t/p/w600_and_h900_bestv2/39lXk6ud6KiJgGbbWI2PUKS7y2.jpg
Distance to query: 0.502

Die Hard 2 1990 1573 https://image.tmdb.org/t/p/w600_and_h900_bestv2/2rto4EhQrUsJoGGgWcwiRv6hB3H.jpg
Distance to query: 0.506



In [10]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="overview",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

Home Alone 1990 771 https://image.tmdb.org/t/p/w600_and_h900_bestv2/onTSipZ8R3bliBdKfPtsDuHTdlL.jpg
Distance to query: 0.414

Sleepless in Seattle 1993 858 https://image.tmdb.org/t/p/w600_and_h900_bestv2/iLWsLVrfkFvOXOG9PbUAYg7AK3E.jpg
Distance to query: 0.430

Home Alone 2: Lost in New York 1992 772 https://image.tmdb.org/t/p/w600_and_h900_bestv2/uuitWHpJwxD1wruFl2nZHIb4UGN.jpg
Distance to query: 0.444

Scent of a Woman 1992 9475 https://image.tmdb.org/t/p/w600_and_h900_bestv2/f75Fsfx8nA8JfSdJ61jhC8JwHps.jpg
Distance to query: 0.444

The Nightmare Before Christmas 1993 9479 https://image.tmdb.org/t/p/w600_and_h900_bestv2/oQffRNjK8e19rF7xVYEN8ew0j7b.jpg
Distance to query: 0.454



In [ ]:
client.close()